문제02_타이타닉.txt
타이타닉 데이터의 생존여부를 예측(분류)하는 모델을 생성하세요.  
설명변수로, 나이, 성별, 객실 등급을 사용합니다. 
전처리 과정을 수행하세요. 
(예 - 결측치, 이상치 처리,  원핫인코딩 등) 
학습 데이터와 테스트 데이터로 분류하여 
테스트 데이터로 재현율, 정밀도, 정확도로 평가하세요.   

RandomForestClassifier 로 학습한 모델과 
SVC 로 학습한 모델을 비교하세요. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import  svm, metrics
from sklearn.preprocessing import OneHotEncoder

In [2]:
tt = pd.read_csv('titanic.csv', encoding='euc-kr', usecols=['생존여부', '객실등급','성별','나이'])
tt.head()

,생존여부,객실등급,성별,나이
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0


# 전처리 과정

## 결측치 처리

In [3]:
tt.isna().sum()
tt.dropna(how='any', inplace=True)

## 원핫 인코딩

In [4]:
def one_hot_encoding(col_name):
    enc = OneHotEncoder(sparse_output=False)
    enc.fit(tt[col_name].values.reshape(-1, 1))
    encoded = enc.transform(tt[col_name].values.reshape(-1,1))
    df = pd.DataFrame(encoded, columns=enc.categories_[0])
    return df

In [5]:
onehot_gen=one_hot_encoding('성별')
onehot_cls=one_hot_encoding('객실등급')

In [6]:
tt= tt.reset_index()
tt.drop(columns=['index'])

,생존여부,객실등급,성별,나이
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0
...,...,...,...,...
709,0,3,female,39.0
710,0,2,male,27.0
711,1,1,female,19.0
712,1,1,male,26.0


In [7]:
tt_new =pd.concat([tt,onehot_gen, onehot_cls], axis = 1)

In [8]:
tt_new = tt_new.drop(columns=['index'])

In [9]:
tt_new.head()

,생존여부,객실등급,성별,나이,female,male,1,2,3
0,0,3,male,22.0,0.0,1.0,0.0,0.0,1.0
1,1,1,female,38.0,1.0,0.0,1.0,0.0,0.0
2,1,3,female,26.0,1.0,0.0,0.0,0.0,1.0
3,1,1,female,35.0,1.0,0.0,1.0,0.0,0.0
4,0,3,male,35.0,0.0,1.0,0.0,0.0,1.0


# 데이터 생성

In [10]:
data = []
label = []
for row in tt_new.values:
    age = row[3]/ 100
    data.append([age,row[4],row[5],row[6],row[7],row[8]])
    label.append(row[0])
data_train, data_test, label_train, label_test = train_test_split(data, label)

# SVC

In [11]:
clf = svm.SVC()
clf.fit(data_train, label_train)
# 데이터 예측하기
predict = clf.predict(data_test)

In [12]:
ac_score = metrics.accuracy_score(label_test, predict)
cl_report = metrics.classification_report(label_test, predict)
print("정답률 =", ac_score)
print("리포트 =\n", cl_report)

정답률 = 0.7821229050279329
리포트 =
               precision    recall  f1-score   support

           0       0.73      0.99      0.84       106
           1       0.97      0.48      0.64        73

    accuracy                           0.78       179
   macro avg       0.85      0.74      0.74       179
weighted avg       0.83      0.78      0.76       179



# RandomForest

In [13]:
clf = RandomForestClassifier()
clf.fit(data_train, label_train)
predict = clf.predict(data_test)

In [14]:
ac_score = metrics.accuracy_score(label_test, predict)
cl_report = metrics.classification_report(label_test, predict)
print("정답률 =", ac_score)
print("리포트 =\n", cl_report)

정답률 = 0.7821229050279329
리포트 =
               precision    recall  f1-score   support

           0       0.78      0.88      0.83       106
           1       0.78      0.64      0.71        73

    accuracy                           0.78       179
   macro avg       0.78      0.76      0.77       179
weighted avg       0.78      0.78      0.78       179

